# Oil & Gas Synthetic Data Exploration

This notebook explores the comprehensive synthetic datasets generated for oil and gas company dashboards.

## Dataset Overview

The synthetic data includes:
1. **Geospatial Dataset**: Oil wells, refineries, and facilities with production metrics
2. **Time Series Datasets**: Production trends, pricing, demand forecasting, maintenance, and weather data

**Project**: oil_gas_dashboards  
**Data Quality Score**: 0.892  
**MLflow Tracking**: http://localhost:8768

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from pathlib import Path
import sys
import json
from datetime import datetime, timedelta

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Add scripts to path for data configuration
sys.path.insert(0, '/mnt/code')
from scripts.data_config import get_data_paths

print("📊 Oil & Gas Synthetic Data Exploration")
print("=" * 50)

## 1. Load Datasets

Load all generated datasets using the proper data configuration paths.

In [ ]:
# Get correct data paths
project_name = 'oil_gas_dashboards'
paths = get_data_paths(project_name)
data_dir = paths['base_data_path']
artifacts_dir = paths['artifacts_path']

print(f"Data directory: {data_dir}")
print(f"Artifacts directory: {artifacts_dir}")
print(f"Project type: {'Git-based' if paths['is_git_based'] else 'DFS-based'}")

# Load all datasets
datasets = {}

# Load geospatial data
geospatial_path = data_dir / 'geospatial_facilities.parquet'
if geospatial_path.exists():
    datasets['geospatial'] = pd.read_parquet(geospatial_path)
    print(f"✅ Loaded geospatial data: {len(datasets['geospatial']):,} facilities")
else:
    print(f"❌ Geospatial data not found at {geospatial_path}")

# Load time series datasets
time_series_files = [
    'production_timeseries.parquet',
    'prices_timeseries.parquet', 
    'demand_timeseries.parquet',
    'maintenance_timeseries.parquet',
    'weather_timeseries.parquet'
]

for file in time_series_files:
    dataset_name = file.replace('_timeseries.parquet', '')
    file_path = data_dir / file
    
    if file_path.exists():
        datasets[dataset_name] = pd.read_parquet(file_path)
        
        # Convert date columns to datetime if needed
        if 'date' in datasets[dataset_name].columns:
            datasets[dataset_name]['date'] = pd.to_datetime(datasets[dataset_name]['date'])
        
        print(f"✅ Loaded {dataset_name} data: {len(datasets[dataset_name]):,} records")
    else:
        print(f"❌ {dataset_name} data not found at {file_path}")

print(f"\n📈 Total datasets loaded: {len(datasets)}")

## 2. Geospatial Facilities Analysis

Explore the geographic distribution of oil and gas facilities worldwide.

In [ ]:
if 'geospatial' in datasets:
    geo_df = datasets['geospatial']
    
    print("🌍 Geospatial Dataset Overview")
    print("-" * 40)
    print(f"Total facilities: {len(geo_df):,}")
    print(f"Regions covered: {geo_df['region'].nunique()}")
    print(f"Countries covered: {geo_df['country'].nunique()}")
    print(f"Date range: {pd.Timestamp.now().strftime('%Y-%m-%d')}")
    print()
    
    # Facility type breakdown
    print("📊 Facility Type Distribution:")
    facility_counts = geo_df['facility_type'].value_counts()
    for facility_type, count in facility_counts.items():
        percentage = (count / len(geo_df)) * 100
        print(f"  {facility_type}: {count:,} ({percentage:.1f}%)")
    print()
    
    # Regional distribution
    print("🗺️ Regional Distribution:")
    regional_counts = geo_df['region'].value_counts()
    for region, count in regional_counts.items():
        percentage = (count / len(geo_df)) * 100
        print(f"  {region}: {count:,} ({percentage:.1f}%)")
    print()
    
    # Status analysis
    print("⚡ Facility Status:")
    status_counts = geo_df['status'].value_counts()
    for status, count in status_counts.items():
        percentage = (count / len(geo_df)) * 100
        print(f"  {status}: {count:,} ({percentage:.1f}%)")

In [ ]:
# Create visualizations for geospatial data
if 'geospatial' in datasets:
    geo_df = datasets['geospatial']
    
    # Create subplots
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Facility types pie chart
    facility_counts = geo_df['facility_type'].value_counts()
    axes[0, 0].pie(facility_counts.values, labels=facility_counts.index, autopct='%1.1f%%', startangle=90)
    axes[0, 0].set_title('Facility Type Distribution')
    
    # 2. Regional distribution bar chart
    regional_counts = geo_df['region'].value_counts()
    axes[0, 1].bar(range(len(regional_counts)), regional_counts.values)
    axes[0, 1].set_title('Facilities by Region')
    axes[0, 1].set_xlabel('Region')
    axes[0, 1].set_ylabel('Number of Facilities')
    axes[0, 1].set_xticks(range(len(regional_counts)))
    axes[0, 1].set_xticklabels(regional_counts.index, rotation=45, ha='right')
    
    # 3. Oil production distribution (for oil wells only)
    oil_wells = geo_df[geo_df['facility_type'] == 'oil_well']
    if len(oil_wells) > 0 and 'oil_production_bpd' in oil_wells.columns:
        axes[1, 0].hist(oil_wells['oil_production_bpd'], bins=50, alpha=0.7, edgecolor='black')
        axes[1, 0].set_title('Oil Production Distribution (Wells)')
        axes[1, 0].set_xlabel('Production (barrels/day)')
        axes[1, 0].set_ylabel('Number of Wells')
    
    # 4. Equipment health score distribution
    if 'equipment_health_score' in geo_df.columns:
        axes[1, 1].hist(geo_df['equipment_health_score'], bins=30, alpha=0.7, edgecolor='black')
        axes[1, 1].set_title('Equipment Health Score Distribution')
        axes[1, 1].set_xlabel('Health Score')
        axes[1, 1].set_ylabel('Number of Facilities')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Interactive world map of facilities
if 'geospatial' in datasets:
    geo_df = datasets['geospatial']
    
    # Create interactive map with plotly
    fig = px.scatter_mapbox(
        geo_df.sample(min(500, len(geo_df))),  # Limit to 500 points for performance
        lat="latitude",
        lon="longitude",
        color="facility_type",
        size="oil_production_bpd" if 'oil_production_bpd' in geo_df.columns else None,
        hover_data=["facility_name", "region", "country", "status"],
        mapbox_style="open-street-map",
        zoom=1,
        title="Oil & Gas Facilities Worldwide",
        height=600
    )
    
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
    fig.show()
    
    print(f"📍 Map shows sample of {min(500, len(geo_df))} facilities from {len(geo_df)} total")

## 3. Production Time Series Analysis

Analyze oil and gas production trends over time.

In [ ]:
if 'production' in datasets:
    prod_df = datasets['production']
    
    print("🛢️ Production Dataset Overview")
    print("-" * 40)
    print(f"Total records: {len(prod_df):,}")
    print(f"Date range: {prod_df['date'].min().strftime('%Y-%m-%d')} to {prod_df['date'].max().strftime('%Y-%m-%d')}")
    print(f"Unique wells: {prod_df['facility_id'].nunique():,}")
    print(f"Regions: {prod_df['region'].nunique()}")
    print()
    
    # Aggregate daily production by region
    daily_production = prod_df.groupby(['date', 'region']).agg({
        'oil_production_bpd': 'sum',
        'gas_production_mcfd': 'sum'
    }).reset_index()
    
    print("📈 Average Daily Production by Region:")
    regional_avg = prod_df.groupby('region').agg({
        'oil_production_bpd': 'mean',
        'gas_production_mcfd': 'mean'
    }).round(0)
    
    for region in regional_avg.index:
        oil_avg = regional_avg.loc[region, 'oil_production_bpd']
        gas_avg = regional_avg.loc[region, 'gas_production_mcfd']
        print(f"  {region}: Oil {oil_avg:,.0f} bpd, Gas {gas_avg:,.0f} mcfd")

In [ ]:
# Production time series visualizations
if 'production' in datasets:
    prod_df = datasets['production']
    
    # Aggregate monthly production
    prod_df['year_month'] = prod_df['date'].dt.to_period('M')
    monthly_prod = prod_df.groupby(['year_month', 'region']).agg({
        'oil_production_bpd': 'sum',
        'gas_production_mcfd': 'sum'
    }).reset_index()
    monthly_prod['year_month'] = monthly_prod['year_month'].dt.to_timestamp()
    
    # Create subplots for oil and gas production
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Monthly Oil Production by Region', 'Monthly Gas Production by Region'),
        vertical_spacing=0.08
    )
    
    # Oil production by region
    for region in monthly_prod['region'].unique():
        region_data = monthly_prod[monthly_prod['region'] == region]
        fig.add_trace(
            go.Scatter(
                x=region_data['year_month'],
                y=region_data['oil_production_bpd'],
                mode='lines+markers',
                name=f'{region} Oil',
                line=dict(width=2)
            ),
            row=1, col=1
        )
    
    # Gas production by region
    for region in monthly_prod['region'].unique():
        region_data = monthly_prod[monthly_prod['region'] == region]
        fig.add_trace(
            go.Scatter(
                x=region_data['year_month'],
                y=region_data['gas_production_mcfd'],
                mode='lines+markers',
                name=f'{region} Gas',
                line=dict(width=2),
                showlegend=False
            ),
            row=2, col=1
        )
    
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Oil Production (bpd)", row=1, col=1)
    fig.update_yaxes(title_text="Gas Production (mcfd)", row=2, col=1)
    
    fig.update_layout(
        height=800,
        title_text="Production Trends by Region",
        hovermode='x unified'
    )
    
    fig.show()

## 4. Price Analysis

Examine oil and gas price trends and volatility.

In [ ]:
if 'prices' in datasets:
    price_df = datasets['prices']
    
    print("💰 Price Dataset Overview")
    print("-" * 40)
    print(f"Total records: {len(price_df):,}")
    print(f"Date range: {price_df['date'].min().strftime('%Y-%m-%d')} to {price_df['date'].max().strftime('%Y-%m-%d')}")
    print()
    
    # Calculate price statistics
    price_columns = [col for col in price_df.columns if 'price' in col.lower()]
    
    print("📊 Price Statistics (USD):")
    for col in price_columns:
        avg_price = price_df[col].mean()
        min_price = price_df[col].min()
        max_price = price_df[col].max()
        volatility = price_df[col].std()
        
        product = col.replace('_price_usd_', ' ').replace('_', ' ').title()
        print(f"  {product}: Avg ${avg_price:.2f}, Range ${min_price:.2f}-${max_price:.2f}, Volatility ${volatility:.2f}")

In [ ]:
# Price trends visualization
if 'prices' in datasets:
    price_df = datasets['prices']
    
    # Create interactive price chart
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Crude Oil Prices', 'Refined Product Prices'),
        vertical_spacing=0.08
    )
    
    # Crude oil prices
    oil_price_cols = ['crude_oil_price_usd_bbl', 'brent_crude_usd_bbl', 'wti_crude_usd_bbl']
    for col in oil_price_cols:
        if col in price_df.columns:
            fig.add_trace(
                go.Scatter(
                    x=price_df['date'],
                    y=price_df[col],
                    mode='lines',
                    name=col.replace('_', ' ').title().replace('Usd', 'USD'),
                    line=dict(width=2)
                ),
                row=1, col=1
            )
    
    # Refined product prices
    product_price_cols = ['gasoline_price_usd_gal', 'diesel_price_usd_gal', 'jet_fuel_price_usd_gal']
    for col in product_price_cols:
        if col in price_df.columns:
            fig.add_trace(
                go.Scatter(
                    x=price_df['date'],
                    y=price_df[col],
                    mode='lines',
                    name=col.replace('_', ' ').title().replace('Usd', 'USD'),
                    line=dict(width=2),
                    showlegend=True
                ),
                row=2, col=1
            )
    
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price (USD/barrel)", row=1, col=1)
    fig.update_yaxes(title_text="Price (USD/gallon)", row=2, col=1)
    
    fig.update_layout(
        height=800,
        title_text="Oil and Gas Price Trends",
        hovermode='x unified'
    )
    
    fig.show()

## 5. Demand Forecasting Analysis

Examine regional demand patterns and trends.

In [ ]:
if 'demand' in datasets:
    demand_df = datasets['demand']
    
    print("📊 Demand Dataset Overview")
    print("-" * 40)
    print(f"Total records: {len(demand_df):,}")
    print(f"Date range: {demand_df['date'].min().strftime('%Y-%m-%d')} to {demand_df['date'].max().strftime('%Y-%m-%d')}")
    print(f"Regions: {demand_df['region'].nunique()}")
    print()
    
    # Calculate average demand by region
    avg_demand = demand_df.groupby('region').agg({
        'oil_demand_thousand_bpd': 'mean',
        'gas_demand_thousand_mcfd': 'mean',
        'gasoline_demand_thousand_bpd': 'mean',
        'diesel_demand_thousand_bpd': 'mean'
    }).round(1)
    
    print("📈 Average Daily Demand by Region (thousands):")
    for region in avg_demand.index:
        oil = avg_demand.loc[region, 'oil_demand_thousand_bpd']
        gas = avg_demand.loc[region, 'gas_demand_thousand_mcfd']
        gasoline = avg_demand.loc[region, 'gasoline_demand_thousand_bpd']
        diesel = avg_demand.loc[region, 'diesel_demand_thousand_bpd']
        print(f"  {region}:")
        print(f"    Oil: {oil:,.1f}k bpd, Gas: {gas:,.1f}k mcfd")
        print(f"    Gasoline: {gasoline:,.1f}k bpd, Diesel: {diesel:,.1f}k bpd")

In [ ]:
# Demand trends visualization
if 'demand' in datasets:
    demand_df = datasets['demand']
    
    # Aggregate monthly demand
    demand_df['year_month'] = demand_df['date'].dt.to_period('M')
    monthly_demand = demand_df.groupby(['year_month', 'region']).agg({
        'oil_demand_thousand_bpd': 'mean',
        'gas_demand_thousand_mcfd': 'mean'
    }).reset_index()
    monthly_demand['year_month'] = monthly_demand['year_month'].dt.to_timestamp()
    
    # Create demand trend visualization
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Oil Demand by Region', 'Gas Demand by Region'),
        horizontal_spacing=0.1
    )
    
    # Oil demand
    for region in monthly_demand['region'].unique():
        region_data = monthly_demand[monthly_demand['region'] == region]
        fig.add_trace(
            go.Scatter(
                x=region_data['year_month'],
                y=region_data['oil_demand_thousand_bpd'],
                mode='lines+markers',
                name=f'{region}',
                line=dict(width=2)
            ),
            row=1, col=1
        )
    
    # Gas demand
    for region in monthly_demand['region'].unique():
        region_data = monthly_demand[monthly_demand['region'] == region]
        fig.add_trace(
            go.Scatter(
                x=region_data['year_month'],
                y=region_data['gas_demand_thousand_mcfd'],
                mode='lines+markers',
                name=f'{region} Gas',
                line=dict(width=2),
                showlegend=False
            ),
            row=1, col=2
        )
    
    fig.update_xaxes(title_text="Date")
    fig.update_yaxes(title_text="Oil Demand (thousand bpd)", row=1, col=1)
    fig.update_yaxes(title_text="Gas Demand (thousand mcfd)", row=1, col=2)
    
    fig.update_layout(
        height=600,
        title_text="Regional Demand Trends",
        hovermode='x unified'
    )
    
    fig.show()

## 6. Maintenance Analysis

Examine equipment maintenance patterns and costs.

In [ ]:
if 'maintenance' in datasets:
    maint_df = datasets['maintenance']
    
    print("🔧 Maintenance Dataset Overview")
    print("-" * 40)
    print(f"Total maintenance events: {len(maint_df):,}")
    print(f"Date range: {maint_df['date'].min().strftime('%Y-%m-%d')} to {maint_df['date'].max().strftime('%Y-%m-%d')}")
    print(f"Unique facilities: {maint_df['facility_id'].nunique():,}")
    print()
    
    # Maintenance type analysis
    print("🛠️ Maintenance Type Distribution:")
    maint_types = maint_df['maintenance_type'].value_counts()
    total_cost = maint_df['cost_usd'].sum()
    
    for mtype, count in maint_types.items():
        percentage = (count / len(maint_df)) * 100
        type_cost = maint_df[maint_df['maintenance_type'] == mtype]['cost_usd'].sum()
        cost_percentage = (type_cost / total_cost) * 100
        avg_cost = maint_df[maint_df['maintenance_type'] == mtype]['cost_usd'].mean()
        print(f"  {mtype}: {count:,} events ({percentage:.1f}%), Avg cost: ${avg_cost:,.0f}")
    
    print(f"\n💰 Total maintenance cost: ${total_cost:,.0f}")
    print(f"📅 Average cost per day: ${total_cost / len(maint_df['date'].unique()):,.0f}")

In [ ]:
# Maintenance analysis visualizations
if 'maintenance' in datasets:
    maint_df = datasets['maintenance']
    
    # Create maintenance analysis plots
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Maintenance type distribution
    maint_types = maint_df['maintenance_type'].value_counts()
    axes[0, 0].pie(maint_types.values, labels=maint_types.index, autopct='%1.1f%%', startangle=90)
    axes[0, 0].set_title('Maintenance Type Distribution')
    
    # 2. Cost by maintenance type
    cost_by_type = maint_df.groupby('maintenance_type')['cost_usd'].sum().sort_values(ascending=False)
    axes[0, 1].bar(range(len(cost_by_type)), cost_by_type.values)
    axes[0, 1].set_title('Total Maintenance Cost by Type')
    axes[0, 1].set_xlabel('Maintenance Type')
    axes[0, 1].set_ylabel('Total Cost (USD)')
    axes[0, 1].set_xticks(range(len(cost_by_type)))
    axes[0, 1].set_xticklabels(cost_by_type.index, rotation=45, ha='right')
    
    # 3. Duration distribution
    axes[1, 0].hist(maint_df['duration_hours'], bins=30, alpha=0.7, edgecolor='black')
    axes[1, 0].set_title('Maintenance Duration Distribution')
    axes[1, 0].set_xlabel('Duration (hours)')
    axes[1, 0].set_ylabel('Number of Events')
    
    # 4. Cost distribution
    axes[1, 1].hist(maint_df['cost_usd'], bins=50, alpha=0.7, edgecolor='black')
    axes[1, 1].set_title('Maintenance Cost Distribution')
    axes[1, 1].set_xlabel('Cost (USD)')
    axes[1, 1].set_ylabel('Number of Events')
    
    plt.tight_layout()
    plt.show()

## 7. Weather Impact Analysis

Examine weather conditions and their impact on operations.

In [ ]:
if 'weather' in datasets:
    weather_df = datasets['weather']
    
    print("🌤️ Weather Dataset Overview")
    print("-" * 40)
    print(f"Total records: {len(weather_df):,}")
    print(f"Date range: {weather_df['date'].min().strftime('%Y-%m-%d')} to {weather_df['date'].max().strftime('%Y-%m-%d')}")
    print(f"Regions: {weather_df['region'].nunique()}")
    print()
    
    # Weather condition analysis
    print("🌦️ Weather Condition Distribution:")
    weather_conditions = weather_df['weather_condition'].value_counts()
    for condition, count in weather_conditions.items():
        percentage = (count / len(weather_df)) * 100
        avg_impact = weather_df[weather_df['weather_condition'] == condition]['operation_impact_score'].mean()
        print(f"  {condition}: {count:,} days ({percentage:.1f}%), Avg impact: {avg_impact:.3f}")
    
    print()
    
    # Regional climate summary
    print("🌡️ Regional Climate Summary:")
    climate_summary = weather_df.groupby('region').agg({
        'temperature_celsius': ['mean', 'min', 'max'],
        'humidity_percent': 'mean',
        'precipitation_mm': 'mean',
        'operation_impact_score': 'mean'
    }).round(1)
    
    for region in climate_summary.index:
        temp_avg = climate_summary.loc[region, ('temperature_celsius', 'mean')]
        temp_min = climate_summary.loc[region, ('temperature_celsius', 'min')]
        temp_max = climate_summary.loc[region, ('temperature_celsius', 'max')]
        humidity = climate_summary.loc[region, ('humidity_percent', 'mean')]
        precipitation = climate_summary.loc[region, ('precipitation_mm', 'mean')]
        impact = climate_summary.loc[region, ('operation_impact_score', 'mean')]
        
        print(f"  {region}:")
        print(f"    Temperature: {temp_avg:.1f}°C (range: {temp_min:.1f}°C to {temp_max:.1f}°C)")
        print(f"    Humidity: {humidity:.1f}%, Precipitation: {precipitation:.1f}mm/day")
        print(f"    Operation impact: {impact:.3f}")

In [ ]:
# Weather impact visualization
if 'weather' in datasets:
    weather_df = datasets['weather']
    
    # Monthly temperature trends by region
    weather_df['year_month'] = weather_df['date'].dt.to_period('M')
    monthly_weather = weather_df.groupby(['year_month', 'region']).agg({
        'temperature_celsius': 'mean',
        'operation_impact_score': 'mean'
    }).reset_index()
    monthly_weather['year_month'] = monthly_weather['year_month'].dt.to_timestamp()
    
    # Create temperature trends chart
    fig = px.line(
        monthly_weather,
        x='year_month',
        y='temperature_celsius',
        color='region',
        title='Monthly Temperature Trends by Region',
        labels={'temperature_celsius': 'Temperature (°C)', 'year_month': 'Date'},
        height=500
    )
    
    fig.update_layout(hovermode='x unified')
    fig.show()
    
    # Operation impact vs weather conditions
    fig = px.box(
        weather_df,
        x='weather_condition',
        y='operation_impact_score',
        title='Operation Impact by Weather Condition',
        labels={'operation_impact_score': 'Operation Impact Score', 'weather_condition': 'Weather Condition'},
        height=400
    )
    
    fig.update_xaxes(tickangle=45)
    fig.show()

## 8. Data Quality Assessment

Review the quality and completeness of generated datasets.

In [ ]:
# Load data quality summary from generation
summary_path = artifacts_dir / 'reports' / 'data_generation_summary.json'

if summary_path.exists():
    with open(summary_path, 'r') as f:
        summary = json.load(f)
    
    print("✅ Data Quality Assessment")
    print("=" * 50)
    print(f"Generation Date: {summary['generation_date']}")
    print(f"Overall Quality Score: {summary['overall_quality']:.3f}")
    print()
    
    print("📊 Dataset Quality Scores:")
    for dataset, score in summary['quality_scores'].items():
        status = "✅ Excellent" if score >= 0.9 else "⚠️ Good" if score >= 0.7 else "❌ Needs Review"
        print(f"  {dataset}: {score:.3f} {status}")
    
    print()
    print("📈 Dataset Statistics:")
    print(f"  Total facilities: {summary['total_facilities']:,}")
    
    facility_breakdown = summary['facility_breakdown']
    print("  Facility breakdown:")
    for facility_type, count in facility_breakdown.items():
        print(f"    {facility_type}: {count:,}")
    
    print(f"  Regions covered: {', '.join(summary['regions_covered'])}")
    print(f"  Time series datasets: {', '.join(summary['time_series_datasets'])}")
    
else:
    print("❌ Data generation summary not found")

# Additional quality checks on loaded datasets
print("\n🔍 Additional Data Quality Checks:")
print("-" * 40)

for dataset_name, df in datasets.items():
    print(f"\n{dataset_name.title()} Dataset:")
    
    # Missing values
    missing_count = df.isnull().sum().sum()
    missing_percentage = (missing_count / (len(df) * len(df.columns))) * 100
    print(f"  Missing values: {missing_count:,} ({missing_percentage:.2f}%)")
    
    # Duplicate records
    duplicates = df.duplicated().sum()
    print(f"  Duplicate records: {duplicates:,}")
    
    # Data types
    numeric_cols = len(df.select_dtypes(include=[np.number]).columns)
    text_cols = len(df.select_dtypes(include=['object']).columns)
    date_cols = len(df.select_dtypes(include=['datetime64']).columns)
    print(f"  Column types: {numeric_cols} numeric, {text_cols} text, {date_cols} datetime")
    
    # Data range validation
    if dataset_name == 'geospatial':
        lat_range = (df['latitude'].min(), df['latitude'].max())
        lon_range = (df['longitude'].min(), df['longitude'].max())
        print(f"  Geographic range: Lat {lat_range[0]:.2f} to {lat_range[1]:.2f}, Lon {lon_range[0]:.2f} to {lon_range[1]:.2f}")
    
    elif 'date' in df.columns:
        date_range = (df['date'].min().strftime('%Y-%m-%d'), df['date'].max().strftime('%Y-%m-%d'))
        print(f"  Date range: {date_range[0]} to {date_range[1]}")
        
        # Check for data gaps in time series
        expected_days = (df['date'].max() - df['date'].min()).days + 1
        unique_days = df['date'].nunique()
        if dataset_name in ['production', 'prices', 'weather', 'demand']:
            completeness = (unique_days / expected_days) * 100
            print(f"  Temporal completeness: {completeness:.1f}% ({unique_days}/{expected_days} days)")

## 9. Summary and Recommendations

Summary of generated datasets and recommendations for dashboard development.

In [ ]:
print("📋 Synthetic Data Generation Summary")
print("=" * 50)

print("✅ Successfully Generated Datasets:")
total_records = sum(len(df) for df in datasets.values())
print(f"  Total records across all datasets: {total_records:,}")

for dataset_name, df in datasets.items():
    print(f"  {dataset_name.title()}: {len(df):,} records, {len(df.columns)} columns")

if 'geospatial' in datasets:
    geo_df = datasets['geospatial']
    print(f"\n🌍 Geographic Coverage:")
    print(f"  {geo_df['region'].nunique()} regions: {', '.join(geo_df['region'].unique())}")
    print(f"  {geo_df['country'].nunique()} countries")
    print(f"  {len(geo_df)} facilities total")

if any('date' in df.columns for df in datasets.values()):
    time_series_datasets = [name for name, df in datasets.items() if 'date' in df.columns]
    date_ranges = [(datasets[name]['date'].min(), datasets[name]['date'].max()) for name in time_series_datasets]
    overall_start = min(dr[0] for dr in date_ranges)
    overall_end = max(dr[1] for dr in date_ranges)
    
    print(f"\n📅 Temporal Coverage:")
    print(f"  Date range: {overall_start.strftime('%Y-%m-%d')} to {overall_end.strftime('%Y-%m-%d')}")
    print(f"  Duration: {(overall_end - overall_start).days} days")
    print(f"  Time series datasets: {len(time_series_datasets)}")

print("\n🎯 Dashboard Development Recommendations:")
print("-" * 50)

recommendations = [
    "🗺️ **Geospatial Dashboard**: Create interactive maps showing facility locations, production levels, and equipment health",
    "📊 **Production Dashboard**: Build time series visualizations for oil/gas production trends by region and facility",
    "💰 **Financial Dashboard**: Develop price tracking and revenue analytics using price and production data", 
    "📈 **Demand Forecasting**: Create predictive analytics dashboard using demand patterns and historical trends",
    "🔧 **Maintenance Dashboard**: Build maintenance scheduling and cost optimization tools",
    "🌤️ **Operations Weather Dashboard**: Create weather impact analysis and operational risk assessment",
    "🚨 **Alert System**: Implement real-time monitoring for equipment health, production anomalies, and extreme weather",
    "📋 **Executive Summary**: Build high-level KPI dashboard combining all data sources for leadership overview"
]

for i, recommendation in enumerate(recommendations, 1):
    print(f"{i}. {recommendation}")

print("\n🛠️ Technical Integration Points:")
print("-" * 40)
print("• Use Plotly/Dash for interactive Python dashboards")
print("• Implement real-time data refresh using Domino scheduled jobs")
print("• Deploy as Domino Apps for enterprise access")
print("• Use MLflow for model tracking and deployment")
print("• Implement data quality monitoring and alerting")
print("• Create API endpoints for dashboard data serving")

print("\n📁 File Locations:")
print("-" * 20)
print(f"📂 Data: {data_dir}")
print(f"📂 Artifacts: {artifacts_dir}")
print(f"📂 Notebooks: /mnt/code/notebooks/")
print(f"📂 Scripts: /mnt/code/scripts/")
print(f"🌐 MLflow UI: http://localhost:8768")

print("\n✨ Data generation completed successfully!")
print("Ready for dashboard development and deployment on Domino Data Lab platform.")